In [1]:
import pandas as pd
import os
from datetime import date, timedelta
import datetime
import numpy as np
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from statsmodels.tsa.statespace.sarimax import SARIMAX
# grid search sarima hyperparameters for monthly mean temp dataset
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

In [2]:
ROOT_DIR = os.path.realpath(os.path.join(os.getcwd(), '..'))
cln_pkl_loc = os.path.join(ROOT_DIR,'plankhee','AAcleaned.pkl')

In [3]:
df=pd.read_pickle(cln_pkl_loc)


In [4]:
# one-step sarima forecast
def sarima_forecast(history, config):
	order, sorder, trend = config
	# define model
	model = SARIMAX(history, order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
	# fit model
	model_fit = model.fit(disp=False)
	# make one step forecast
	yhat = model_fit.predict(len(history), len(history))
	return yhat[0]
 
# root mean squared error or rmse
def measure_rmse(actual, predicted):
	return sqrt(mean_squared_error(actual, predicted))
 
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]
 
# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
	#print('starting walk_forward_validation')
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# fit model and make forecast for history
		yhat = sarima_forecast(history, cfg)
		# store forecast in list of predictions
		predictions.append(yhat)  
		# add actual observation to history for the next loop
		history.append(test[i])
	# estimate prediction error
	error = measure_rmse(test, predictions)
	#print(error)
	#print('ending grid_search')
	return error
 
# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
	#print('starting score_model')
	result = None
	# convert config to a key
	key = str(cfg)
	# show all warnings and fail on exception if debugging
	if debug:
		result = walk_forward_validation(data, n_test, cfg)
	else:
		# one failure during model validation suggests an unstable config
		try:
			# never show warnings when grid searching, too noisy
			with catch_warnings():
				filterwarnings("ignore")
				result = walk_forward_validation(data, n_test, cfg)
		except:
			error = None
	# check for an interesting result
	#if result is not None:
		#print(' > Model[%s] %.3f' % (key, result))
	#print('ending grid_search')
	return (key, result)
 
# grid search configs
def grid_search(data, cfg_list, n_test, parallel=True):
	#print('starting grid_search')
	scores = None
	if parallel:
		# execute configs in parallel
		executor = Parallel(n_jobs=cpu_count())#, backend='multiprocessing')
		tasks = (delayed(score_model)(data, n_test, cfg) for cfg in tqdm(cfg_list))
		scores = executor(tasks)
	else:
		scores = [score_model(data, n_test, cfg) for cfg in tqdm(cfg_list)]
	# remove empty results
	scores = [r for r in scores if r[1] != None]
	# sort configs by error, asc
	scores.sort(key=lambda tup: tup[1])
	#print('ending grid_search')
	return scores
 
# create a set of sarima configs to try
def sarima_configs(seasonal=[0]):
	#print('starting sarima_configs')
	models = list()
	# define config lists
	p_params = [0, 1, 2]
	d_params = [0, 1]
	q_params = [0, 1, 2]
	t_params = ['n','c','t','ct']
	P_params = [0, 1, 2]
	D_params = [0, 1]
	Q_params = [0, 1, 2]
	m_params = seasonal
	# create config instances
	for p in p_params:
		for d in d_params:
			for q in q_params:
				for t in t_params:
					for P in P_params:
						for D in D_params:
							for Q in Q_params:
								for m in m_params:
									cfg = [(p,d,q), (P,D,Q,m), t]
									models.append(cfg)
	#print('ending sarima_configs')
	return models

In [5]:
if __name__ == '__main__':
	# load dataset
	series = df.loc['2022-08-21 00:00:00':'2022-08-31 23:00:00']['temp']
	data = series.values
	# trim dataset to 5 years
	#data = data[-(5*12):]
	# data split
	n_test = 24
	# model configs
	cfg_list = sarima_configs(seasonal=[24])
	#cfg_list = [[(0, 0, 0), (0, 0, 0, 0), 'n']]
	# grid search
	scores = grid_search(data, cfg_list, n_test)
	print('done')
	#list top 3 configs
	for cfg, error in scores[:3]:
		print(cfg, error)

100%|██████████| 1296/1296 [1:09:44<00:00,  3.23s/it]


done
[(2, 1, 2), (0, 1, 2, 24), 'n'] 1.082117832572393
[(2, 1, 1), (1, 1, 2, 24), 'n'] 1.0859682373658481
[(0, 1, 0), (1, 1, 2, 24), 't'] 1.0866086180811896


In [11]:
df_score=pd.DataFrame(scores, columns=['config', 'score']) 
df_score.to_csv('SARMIAHyperScores.csv')